In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os 

### Setup (Reloading the Data)

In [44]:
DATA_PATH = '/Users/manarmohsin/ecommerce-cac-optimization/data/raw/archive'

In [45]:
# Load all individual CSV files into a Python dictionary called 'dataframes'

dataframes = {
    'customers': pd.read_csv(os.path.join(DATA_PATH, 'olist_customers_dataset.csv')),
    'orders': pd.read_csv(os.path.join(DATA_PATH, 'olist_orders_dataset.csv')),
    'items': pd.read_csv(os.path.join(DATA_PATH, 'olist_order_items_dataset.csv')),
    'payments': pd.read_csv(os.path.join(DATA_PATH, 'olist_order_payments_dataset.csv')),
    'products': pd.read_csv(os.path.join(DATA_PATH, 'olist_products_dataset.csv')),
    'reviews': pd.read_csv(os.path.join(DATA_PATH, 'olist_order_reviews_dataset.csv')),
    'sellers': pd.read_csv(os.path.join(DATA_PATH, 'olist_sellers_dataset.csv')),
    'category_translation': pd.read_csv(os.path.join(DATA_PATH, 'product_category_name_translation.csv'))
}

print("All necessary files loaded successfully!")

All necessary files loaded successfully!


In [46]:
#Data Merging Block (Sequential Joins)

# Stage 1: Start with the core 'orders' data and link to 'customers'.
df_combined = dataframes['orders'].merge(dataframes['customers'], on= 'customer_id', how='left')

# Stage 2: Merge with Order Items.
df_combined = df_combined.merge(dataframes['items'], on ='order_id', how='left')

# Stage 3: Merge with Payments.
df_combined = df_combined.merge(dataframes['payments'], on='order_id', how='left')

# Stage 4: Merge with Product Details.
df_combined = df_combined.merge(dataframes['products'], on='product_id', how='left')

# Stage 5: Merge with Category Translation.
df_combined = df_combined.merge(dataframes['category_translation'], on='product_category_name', how='left')

# Stage 6: Merge with Reviews.
df_combined = df_combined.merge(dataframes['sellers'], on='seller_id', how='left')


### Missing Values Analysis

In [47]:
#Missing counts
df_combined.isnull().sum().sort_values(ascending=False)

order_delivered_customer_date    3397
product_category_name_english    2553
product_description_lenght       2528
product_category_name            2528
product_photos_qty               2528
product_name_lenght              2528
order_delivered_carrier_date     2074
product_weight_g                  850
product_height_cm                 850
product_length_cm                 850
product_width_cm                  850
seller_zip_code_prefix            830
product_id                        830
freight_value                     830
seller_id                         830
shipping_limit_date               830
order_item_id                     830
price                             830
seller_city                       830
seller_state                      830
order_approved_at                 176
payment_sequential                  3
payment_value                       3
payment_type                        3
payment_installments                3
order_id                            0
customer_id 

In [48]:
#Percentage of missing rows
df_combined.isnull().sum() / len(df_combined) * 100

order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_approved_at                0.148606
order_delivered_carrier_date     1.751186
order_delivered_customer_date    2.868264
order_estimated_delivery_date    0.000000
customer_unique_id               0.000000
customer_zip_code_prefix         0.000000
customer_city                    0.000000
customer_state                   0.000000
order_item_id                    0.700812
product_id                       0.700812
seller_id                        0.700812
shipping_limit_date              0.700812
price                            0.700812
freight_value                    0.700812
payment_sequential               0.002533
payment_type                     0.002533
payment_installments             0.002533
payment_value                    0.002533
product_category_name            2.134522
product_name_lenght              2

### Issue 1: Missing Values Analysis

* **order_delivered_customer_date:** Critical for LTV. Missing data likely means the order was never delivered. Must be investigated via the `order_status`. 
* **product_category_name_english:** Critical for segmentation. Cannot segment the customer's purchase by product type. Will drop the rows if the analysis heavily relies on product segmentation
* **order_delivered_carrier_date:** Critical for devlivery time. missing data mean the item was not shiped by the carrier. Investigate with `order_status`
* **Product Attributes (Length, Weight, etc.):** Low priority. Missing product dimensions/namesn is likely a data entry issue for a few products. Does not affect LTV/CAC analysis. 
* **Seller/Item Details (price, seller_id, freight):** Critical for revenue. These are orders with missing financial and item details. Must be droped becuas ethe represent invalid sales records that cannot be included in any revenue or cost calculation. 
* **Payment Details (payment_value, payment_type):** Only 3 rows missing payment info. Can safely be dropped. 



### Duplicate Analysis

In [49]:
df_combined.duplicated().sum()

np.int64(0)

In [50]:
df_combined.duplicated(subset=['order_id', 'order_item_id']).sum()

np.int64(5009)

### Issue 2: Transaction Duplication due to Multiple Payments

* **Found:** 5,009 rows were identified as duplicates when checking the key combination of `order_id` and `order_item_id`.
* **Possible reason:** This is caused by the relational join with the `order_payments` table. A single order item is duplicated for every payment transaction used to pay for that order (e.g., using two credit cards results in two rows for the same product).
* **Decision:** We cannot simply drop these rows, as they contain valid payment information (`payment_value`). We will need to aggregate the data by `order_id` and `order_item_id` in the next cleaning stage to consolidate payment values (summing the `payment_value` and keeping only one row per item).
* **Business impact:** If not handled, any calculation of total revenue or average price will be inflated due to double-counting the cost of the single item.

### Outlier and Consistency Analysis

In [51]:
df_combined[df_combined['price'] < 0].shape[0]

0

No rows in the entire combined dataset where price is negative

In [52]:
df_combined[df_combined['payment_value'] > 10000].shape[0

8

8 transactions in the dataset with a total greater than R$10,000 (8 outliers)

In [30]:
# finding date inconsistency
df_combined.select_dtypes(include='object')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_city,customer_state,product_id,seller_id,shipping_limit_date,payment_type,product_category_name,product_category_name_english,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,credit_card,utilidades_domesticas,housewares,maua,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,voucher,utilidades_domesticas,housewares,maua,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,voucher,utilidades_domesticas,housewares,maua,SP
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,barreiras,BA,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,boleto,perfumaria,perfumery,belo horizonte,SP
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,GO,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,credit_card,automotivo,auto,guariba,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,praia grande,SP,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,credit_card,bebes,baby,tupa,SP
118430,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,nova vicosa,BA,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,credit_card,eletrodomesticos_2,home_appliances_2,sao paulo,SP
118431,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,japuiba,RJ,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,credit_card,informatica_acessorios,computers_accessories,ilicinea,MG
118432,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,japuiba,RJ,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,credit_card,informatica_acessorios,computers_accessories,ilicinea,MG


In [54]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118434 entries, 0 to 118433
Data columns (total 34 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       118434 non-null  object 
 1   customer_id                    118434 non-null  object 
 2   order_status                   118434 non-null  object 
 3   order_purchase_timestamp       118434 non-null  object 
 4   order_approved_at              118258 non-null  object 
 5   order_delivered_carrier_date   116360 non-null  object 
 6   order_delivered_customer_date  115037 non-null  object 
 7   order_estimated_delivery_date  118434 non-null  object 
 8   customer_unique_id             118434 non-null  object 
 9   customer_zip_code_prefix       118434 non-null  int64  
 10  customer_city                  118434 non-null  object 
 11  customer_state                 118434 non-null  object 
 12  order_item_id                 

### Issue 3: Date Type Inconsistency
Reviewed the df_combined.info() output, which showed several date-related columns were incorrectly stored as the object (string/text) data type (`order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`, and `order_estimated_delivery_date`)

This makes it hard to do some calculations, suc as shipping time, delay, etc. All these columns needs to be converted to the datetime64 format. 

In [35]:
# Formatting and Case Consistency (Categorical Columns)

df_combined['customer_city'] = df_combined['customer_city'].str.lower().str.strip()
df_combined['seller_state'] = df_combined['seller_state'].str.upper().str.strip()
df_combined['order_status'] = df_combined['order_status'].str.lower().str.strip()
df_combined['payment_type'] = df_combined['payment_type'].str.lower().str.strip()



In [37]:
# Select all columns with float or integer data types and generate summary statistics.
numerical_summary = df_combined.select_dtypes(include=['float64', 'int64']).describe().T

# Display the summary table
print(numerical_summary)

                               count          mean           std      min  \
customer_zip_code_prefix    118434.0  35034.264392  29819.290218  1003.00   
order_item_id               117604.0      1.195920      0.697721     1.00   
price                       117604.0    120.822849    184.477367     0.85   
freight_value               117604.0     20.045551     15.861351     0.00   
payment_sequential          118431.0      1.094207      0.728290     1.00   
payment_installments        118431.0      2.937398      2.774164     0.00   
payment_value               118431.0    172.849395    268.259831     0.00   
product_name_lenght         115906.0     48.767208     10.035162     5.00   
product_description_lenght  115906.0    786.957302    653.029009     4.00   
product_photos_qty          115906.0      2.207849      1.719519     1.00   
product_weight_g            117584.0   2114.281067   3788.754048     0.00   
product_length_cm           117584.0     30.256872     16.190874     7.00   

In [42]:
# Check the count of each unique order status
status_counts = df_combined['order_status'].value_counts()

print("--- Order Status Counts ---")
print(status_counts)
print("\n--- Order Status Percentages ---")
print(status_counts / len(df_combined) * 100)

--- Order Status Counts ---
order_status
delivered      115038
shipped          1245
canceled          745
unavailable       650
processing        375
invoiced          373
created             5
approved            3
Name: count, dtype: int64

--- Order Status Percentages ---
order_status
delivered      97.132580
shipped         1.051218
canceled        0.629042
unavailable     0.548829
processing      0.316632
invoiced        0.314943
created         0.004222
approved        0.002533
Name: count, dtype: float64


In [41]:
translated_count = df_combined['product_category_name_english'].nunique()

missing_translation = df_combined['product_category_name_english'].isnull().sum()

print(f"Unique Portuguese Categories: {original_count}")
print(f"Unique English Categories: {translated_count}")
print(f"Rows with Missing English Translation (NaN): {missing_translation}")

# Find which original Portuguese categories are missing a translation
untranslated_categories = df_combined[
    df_combined['product_category_name_english'].isnull()
]['product_category_name'].unique()

print("\nSample of Untranslated Portuguese Categories (should be small list):")
print(untranslated_categories[:5])

Unique Portuguese Categories: 73
Unique English Categories: 71
Rows with Missing English Translation (NaN): 2553

Sample of Untranslated Portuguese Categories (should be small list):
[nan 'portateis_cozinha_e_preparadores_de_alimentos' 'pc_gamer']


### Issue 4: Unit and Scale Consistency (Dimensions)

* **Check:** Used `df_combined.select_dtypes().describe().T` to review the minimum and maximum values for product dimensions (weight, length, height, width).
* **Found:** All dimension columns show a **minimum value $\ge 0$** and a **maximum value that is physically plausible** (e.g., max weight $\approx 40$ kg; max dimension $\approx 105$ cm).
* **Decision:** No evidence of unit mixing (e.g., grams vs. milligrams) or negative entries. No cleaning action is required for these columns.
* **Business Impact:** Accurate dimensions are confirmed, allowing for reliable calculations involving freight and volume.

### Issue 5: Currency Consistency (Financials)

* **Check:** Reviewed the `min` values for `price`, `freight_value`, and `payment_value`.
* **Found:** All minimum values are $\mathbf{\ge R\$0.00}$. No negative financial values were found, indicating high data quality with respect to currency. As all data comes from the Olist platform, we confirm all values are consistently measured in **Brazilian Real (R$)**.
* **Decision:** No currency standardization is needed. The zero-value payments will be resolved by filtering for 'delivered' orders.
* **Business Impact:** Core revenue and cost metrics are protected from negative or mixed-currency errors.


### Issue 6: Product Category Translation Completeness

* **Check:** Compared the count of unique Portuguese categories to unique English categories, and checked for NaN values in `product_category_name_english`.
* **Found:** The count of unique categories is nearly identical (e.g., 73 Portuguese vs. 71 English). However, **2,553 rows have a missing English translation**.
    * Further analysis shows these 2,553 rows are missing the original Portuguese category name entirely (`product_category_name` is also missing for these rows).
* **Decision:** These 2,553 rows represent sales where the product was never correctly categorized. If product analysis is required, these rows must be **removed**. We will track this decision for the cleaning phase.
* **Business Impact:** Removing these rows ensures all segmentation (LTV by category) is accurate.

### Summary of Cleaning Decisions

* **Duplicate Removal:** Aggregate rows based on `order_id` and `order_item_id` to consolidate multiple payments into a single transaction record, fixing the 5,009 duplicates.

* **Incomplete/Failed Orders:** Filter out all orders where `order_status` is NOT 'delivered' (3,396 rows). This simultaneously fixes the missing delivery dates and the issue of zero-value payments.

* **Missing Financial/Item Details:** Drop rows missing critical data like `price`, `freight_value`, `seller_id`, or `product_category_name_english`. (This mainly covers the 830 rows missing item details and the 2,553 rows missing product categories, though the 'delivered' filter will handle some of these).

* **Date Conversion:** Convert all remaining date columns to the datetime64 format.